In [1]:
from bayes_opt import BayesianOptimization, UtilityFunction
import os
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
import numpy as np
from xgboost_learning.hyperparam import BayesSearchParams, get_clean_data,param_transform_pairs
nfiles = 100
ynum = 1
random_state = 3

df,xcols,_ = get_clean_data(nfiles,ynum)
bsp = BayesSearchParams(**param_transform_pairs)
params = {"objective": "reg:squarederror"}
optimizer = BayesianOptimization(f = None, 
                                pbounds = bsp.pbounds, 
                                verbose = 2, \
                                random_state = random_state,\
                                allow_duplicate_points = True)    

params_col = {1:{},2:{}}
bayes_logs_dir = 'bayes_logs'
files = os.listdir(bayes_logs_dir)
for f in files:
    try:
        ynum  = int(f.split('_')[1].replace('y',''))
        random_state  = int(f.split('_')[2].replace('rnd','').replace('.json',''))
    except:
        continue
    optimizer = BayesianOptimization(f = None, pbounds = bsp.pbounds)
    log_file = os.path.join(bayes_logs_dir,f)
    load_logs(optimizer, logs=[log_file])
    logger = JSONLogger(path=log_file,reset=False)
    if not params_col[ynum]:
        old_target = -np.inf
    else:
        old_target =  params_col[ynum]['target']
    if optimizer.max['target'] > old_target:
        params_col[ynum] = optimizer.max
        params_col[ynum]['params'] = bsp(**params_col[ynum]['params'])[1]

In [2]:
params_col

{1: {'target': -12.846519612980206,
  'params': {'gamma': 109.76270078546494,
   'alpha': 0.19549524484259864,
   'colsample_bytree': 0.40064836619643945,
   'eta': 0.015119336467640998,
   'lambda': 0.003482802087028329,
   'min_child_weight': 1696,
   'subsample': 0.27389695952346965,
   'max_depth': 9}},
 2: {'target': -17.66452658803987,
  'params': {'gamma': 190.27945784656063,
   'alpha': 0.04593678963278562,
   'colsample_bytree': 0.10365526739751217,
   'eta': 0.019677729005806537,
   'lambda': 0.023835608729295653,
   'min_child_weight': 5,
   'subsample': 0.24871673035451514,
   'max_depth': 2}}}

In [21]:
import xgboost as xgb
import dask.distributed

cluster = dask.distributed.LocalCluster()
client = dask.distributed.Client(cluster)

/Users/cemgultekin/Documents/clear-street-markets/.env/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57773 instead
  warnings.warn(


In [19]:
ycol = 'Y1'

In [ ]:
dsplit = xgb.dask.DaskDMatrix(client, df[xcols], df[ycol])